In [1]:
from dataclasses import dataclass
import torch
import torch.nn as nn
from torch.nn import functional as F
import math
import time
torch.set_float32_matmul_precision('high')

In [2]:
from torch.distributed import init_process_group, destroy_process_group
from torch.nn.parallel import DistributedDataParallel as DDP
import torch.distributed as dist
import os

In [3]:
ddp = int(os.environ.get('RANK', -1)) != -1 # is this a ddp run?
if ddp:
    # use of DDP atm demands CUDA, we set the device appropriately according to rank
    print("ddp")
    assert torch.cuda.is_available(), "for now i think we need CUDA for DDP"
    init_process_group(backend='nccl')
    ddp_rank = int(os.environ['RANK'])
    ddp_local_rank = int(os.environ['LOCAL_RANK'])
    ddp_world_size = int(os.environ['WORLD_SIZE'])
    device = f'cuda:{ddp_local_rank}'
    torch.cuda.set_device(device)
    master_process = ddp_rank == 0 # this process will do logging, checkpointing etcls
else:
    # vanilla, non-DDP run
    ddp_rank = 0
    ddp_local_rank = 0
    ddp_world_size = 1
    master_process = True
    # attempt to autodetect device
    device = "cpu"
    if torch.cuda.is_available():
        device = "cuda"
    elif hasattr(torch.backends, "mps") and torch.backends.mps.is_available():
        device = "mps"
    print(f"using device: {device}")


using device: cuda


In [4]:
@dataclass
class GPTConfig:
    block_size: int = 1024 # context windows
    vocab_size: int = 50257 # unique tokens
    n_layer: int = 12 # number of blocks
    n_head: int = 12 # n head inside a casual self attention
    n_embd: int = 768 # size of the embedding vector array

In [5]:
class MLP(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.c_fc = nn.Linear(config.n_embd, config.n_embd * 4)
        self.gelu = nn.GELU(approximate='tanh')
        self.c_proj = nn.Linear(config.n_embd * 4, config.n_embd)

    
        self.c_proj.NANOGPT_SCALE_INIT = 1
    
    def forward(self, x):
        # print("Inside MLP")
        x = self.c_fc(x)
        x = self.gelu(x)
        x = self.c_proj(x)
        return x

In [6]:
class CausalSelfAttention(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.c_attn = nn.Linear(config.n_embd, 3 * config.n_embd)
        self.c_proj = nn.Linear(config.n_embd, config.n_embd)
        self.n_head = config.n_head
        self.n_emdb = config.n_embd

        self.c_proj.NANOGPT_SCALE_INIT = 1
        self.register_buffer("bias", torch.tril(torch.ones(config.block_size, config.block_size))
                            .view(1, 1, config.block_size, config.block_size))

    def forward(self, x):
        # print("Inside self attention")
        B, T, C = x.size()

        qkv = self.c_attn(x)
        q, k, v = qkv.split(self.n_emdb, dim=2)
        k = k.view(B, T, self.n_head, C // self.n_head).transpose(1, 2)
        q = q.view(B, T, self.n_head, C // self.n_head).transpose(1, 2)
        v = v.view(B, T, self.n_head, C // self.n_head).transpose(1, 2)

        att = (q @ k.transpose(-2, -1)) * (1.0/math.sqrt(k.size()[-1]))
        att = att.masked_fill(self.bias[:, :, :T, :T] == 0, float('-inf'))
        att = F.softmax(att, dim=-1)
        y = att @ v

        # y = F.scaled_dot_product_attention(q, k, v, is_causal=True)        
        y = y.transpose(1, 2).contiguous().view(B, T, C)
        y = self.c_proj(y)
        return y

In [7]:
class Block(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.ln_1 = nn.LayerNorm(config.n_embd)
        self.attn = CausalSelfAttention(config)
        self.ln_2 = nn.LayerNorm(config.n_embd)
        self.mlp = MLP(config)

    def forward(self, x):
        # print("Inside block ")
        x = x + self.attn(self.ln_1(x))
        x = x + self.mlp(self.ln_2(x))

        return x

In [8]:
class GPT(nn.Module):
    def __init__(self, config):
        super().__init__()

        self.config = config
        self.transformer = nn.ModuleDict(dict(
            wte = nn.Embedding(config.vocab_size, config.n_embd),
            wpe = nn.Embedding(config.block_size, config.n_embd),
            h = nn.ModuleList([Block(config) for _ in range(config.n_layer)]),
            ln_f = nn.LayerNorm(config.n_embd)
        ))
        self.lm_head = nn.Linear(config.n_embd, config.vocab_size, bias=False)

        # if torch.cuda.device_count() > 1:
        #     print(f"Using {torch.cuda.device_count()} GPUs")
        #     self = nn.DataParallel(self)

        #shared weights
        self.transformer.wte.weight = self.lm_head.weight

        self.apply(self._init_weight)
    
    def _init_weight(self, module):
        if isinstance(module, nn.Linear):
            std = 0.02
            if hasattr(module, 'NANOGPT_SCALE_INIT'):
                std *= (2 * self.config.n_layer) ** -0.5
            torch.nn.init.normal_(module.weight, mean=0.0, std=std)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
        
    def configure_optimizer(self, weight_decay, lr, device):
        import inspect
        param_dict = {pn: p for pn, p in self.named_parameters()}
        param_dict = {pn: p for pn, p in param_dict.items() if p.requires_grad}

        decay_params = [p for n, p in param_dict.items() if p.dim() >= 2]
        nodecay_params = [p for n, p in param_dict.items() if p.dim() < 2]

        optim_group = [
            {'params': decay_params, 'weight_decay': weight_decay},
            {'params': nodecay_params, 'weight_decay': 0.0},
        ]

        num_decay_params = sum(p.numel() for p in decay_params)
        num_nodecay_params = sum(p.numel() for p in nodecay_params)

        print(f"Num decayed paramter tensor: {len(decay_params)}, with {num_decay_params:,} params")
        
        print(f"Num non decayed paramter tensor: {len(nodecay_params)}, with {num_nodecay_params:,} params")

        fused_available = 'fused' in inspect.signature(torch.optim.AdamW).parameters
        used_fused = fused_available and 'cuda' in device
        print(f"using fused AdamW: {used_fused}")
        return torch.optim.AdamW(optim_group, lr=lr, betas=(0.9, 0.95), eps=1e-8)
    
    def forward(self, idx, target=None):
        module = self##.module if isinstance(self, nn.DataParallel) else self
        B, T = idx.size()
        # assert T <= module.config.block_size, f"Cannot forward sequence length {T}, block size: {module.config.block_size}"

        device = idx.device

        pos = torch.arange(0, T, dtype=torch.long, device=device)
        pos_embd = module.transformer.wpe(pos)
        tok_embd = module.transformer.wte(idx)
        x = tok_embd + pos_embd

        for block in module.transformer.h:
            x = block(x)

        x = module.transformer.ln_f(x)
        logits = module.lm_head(x)

        B, T, vocab_size = logits.size()
        loss = None
        if target is not None:
        
            loss = F.cross_entropy(logits.view(-1, vocab_size), target.view(-1))

        return logits, loss


    @classmethod
    def from_pretrained(cls, model_type):
        """Load pretrained GPT-2 from hugging face"""

        print("test")
        assert model_type in {'gpt2', 'gpt2-medium', 'gpt2-large', 'gpt2-xl'}
        from transformers import GPT2LMHeadModel
        print("loading weights from pretrained gpt: %s" % model_type)

        config_args = {
            'gpt2':           dict(n_layer=12, n_head=12, n_embd=768),
            'gpt2-medium':    dict(n_layer=24, n_head=16, n_embd=1024),
            'gpt2-large':      dict(n_layer=36, n_head=20, n_embd=1280),
            'gpt2-xl':         dict(n_layer=48, n_head=25, n_embd=1600),
        }[model_type]

        config_args['vocab_size'] = 50257
        config_args['block_size'] = 1024

        config = GPTConfig(**config_args)
        model = GPT(config)

        model_hf = GPT2LMHeadModel.from_pretrained(model_type)
        sd_hf = model_hf.state_dict()

        if isinstance(model, nn.DataParallel):
            sd_hf_modified = {f'module.{k}': v for k, v in sd_hf.items()}
            sd_hf = sd_hf_modified

        sd = model.state_dict()

        sd_keys = sd.keys()
        sd_keys = [k for k in sd_keys if not k.endswith('.attn.bias')]

        sd_keys_hf = sd_hf.keys()
        sd_keys_hf = [k for k in sd_keys_hf if not k.endswith('.attn.masked_bias')]
        sd_keys_hf = [k for k in sd_keys_hf if not k.endswith('.attn.bias')]

        transposed = ['attn.c_attn.weight', 'attn.c_proj.weight', 'mlp.c_fc.weight', 'mlp.c_proj.weight']

        assert len(sd_keys_hf) == len(sd_keys), f"missmatched keys: {len(sd_keys_hf) != len(sd_keys)}"

        for k in sd_keys_hf:
            if any(k.endswith(w) for w in transposed):
                 # Adjust the key for the transposed check if DataParallel is used
                k_orig = k.replace('module.', '') if isinstance(model, nn.DataParallel) else k
                assert sd_hf[k].shape[::-1] == sd[k].shape, f"Shape mismatch on transposed {k_orig}"
                with torch.no_grad():
                    sd[k].copy_(sd_hf[k].t())
            else:
                assert sd_hf[k].shape == sd[k].shape, f"Shape mismatch on {k}"
                with torch.no_grad():
                    sd[k].copy_(sd_hf[k])


        return model

In [9]:
torch.manual_seed(42)
if torch.cuda.is_available():
    torch.manual_seed(1337)
# model = GPT.from_pretrained('gpt2')
model = GPT(GPTConfig())
print("gpt model loaded")

gpt model loaded


In [10]:
num_return_sequences = 1
max_length = 30
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model.eval()
model.to(device)

GPT(
  (transformer): ModuleDict(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (h): ModuleList(
      (0-11): 12 x Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): CausalSelfAttention(
          (c_attn): Linear(in_features=768, out_features=2304, bias=True)
          (c_proj): Linear(in_features=768, out_features=768, bias=True)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): MLP(
          (c_fc): Linear(in_features=768, out_features=3072, bias=True)
          (gelu): GELU(approximate='tanh')
          (c_proj): Linear(in_features=3072, out_features=768, bias=True)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

In [11]:
import tiktoken
enc = tiktoken.get_encoding('gpt2')

tokens = enc.encode("Hello, I am a language model,")
tokens = torch.tensor(tokens, dtype=torch.long)
tokens = tokens.unsqueeze(0).repeat(num_return_sequences, 1)
x = tokens.to(device)

In [12]:
print(f"GPUs ({torch.cuda.device_count()}) available. Running inference...")
torch.manual_seed(42)
torch.cuda.manual_seed(42)
temperature = 0.8
while x.size(1) < max_length:
    with torch.no_grad():
        logits = model(x)
        if isinstance(logits, tuple):
            logits = logits[0]  # Unpack logits from tuple

        logits = logits[:, -1, :]

        probs = F.softmax(logits, dim=-1)
        topk_probs, topk_indices = torch.topk(probs, 50, dim=-1)
        ix = torch.multinomial(topk_probs, 1)
        xcol = torch.gather(topk_indices, -1, ix)
        x = torch.cat((x, xcol), dim=1)
print("Inference finished.")

GPUs (1) available. Running inference...
Inference finished.


In [13]:
for i in range(num_return_sequences):
    tokens = x[i, :max_length].tolist()
    decoded = enc.decode(tokens)
    print("> ", decoded)

>  Hello, I am a language model, competitions competitions modern rad hanged signaled legislaturepriv Peshitative bury imb Beir legislaurus Beirnovaspeak similar colonization Facilities Rebels


In [14]:
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

--2025-06-28 18:12:21--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt’

input.txt           100%[===================>]   1.06M  4.79MB/s    in 0.2s    

2025-06-28 18:12:21 (4.79 MB/s) - ‘input.txt’ saved [1115394/1115394]



In [15]:
class DataLoader:
    def  __init__(self, B, T):
        self.B = B
        self.T = T

        with open('input.txt', 'r') as f:
            text = f.read()

        enc = tiktoken.get_encoding('gpt2')
        tokens = enc.encode(text)
        self.tokens = torch.tensor(tokens)
        print(f"loaded {len(self.tokens)} tokens")
        print(f"1 epoch = {len(self.tokens) // (B * T)} batches")

        self.current = 0
    def next_batch(self):
        B = self.B
        T = self.T
        # Ensure that we have enough tokens for a full batch
        end = min(self.current + B * T + 1, len(self.tokens))
        buf = self.tokens[self.current:end]
        buf = buf.to(device)

        if len(buf) < B * T + 1:
            # If not enough tokens for a full batch, reset and take from the beginning
            self.current = 0
            end = B * T + 1
            buf = self.tokens[self.current:end]
            buf = buf.to(device)


        x = (buf[:-1]).view(B, T)
        y = (buf[1:]).view(B, T)
        self.current += B * T

        if self.current + (B*T + 1) > len(self.tokens):
            print("ROUNDING")
            self.current = 0

        return x, y

In [16]:
total_batch_size = 2**19
B = 4
T = 1024
assert total_batch_size % (B * T) == 0
grad_accum_step = total_batch_size // (B * T)
print(f"total desiderd batch size: {total_batch_size}")
print(f"calcualted gradienet accumulation steps: {grad_accum_step}")

trainloader = DataLoader(B=B, T=T)
x, y = trainloader.next_batch()
x = x.to(device)
y = y.to(device)
model = GPT(GPTConfig(vocab_size=50304))
model.to(device)
torch.compile(model)
logits, loss = model(x, y)
print(loss)

total desiderd batch size: 524288
calcualted gradienet accumulation steps: 128
loaded 338025 tokens
1 epoch = 82 batches
tensor(10.9501, device='cuda:0', grad_fn=<NllLossBackward0>)


In [17]:
max_lr = 6e-4
min_lr = max_lr * 0.1
warmup_step = 10
max_steps = 50

def get_lr(it):
    if it < warmup_step:
        return max_lr * (it + 1) / warmup_step

    if it > max_steps:
        return min_lr

    decay_ratio = (it - warmup_step) / (max_steps - warmup_step)
    assert 0 <= decay_ratio <= 1
    coeff = 0.5 * (1.0 + math.cos(math.pi * decay_ratio))
    return min_lr + coeff * (max_lr - min_lr)

In [ ]:
#optimizer = torch.optim.Adam(model.parameters(), lr=3e-4, betas=(0.9, 0.95), eps=1e-8)
optimizer = model.configure_optimizer(weight_decay=0.1, lr=6e-4, device=device)
print(
    f"{'Step':>6} | {'Loss':>8} | {'LR':>10} | {'Time (ms)':>9} | {'GradNorm':>10} | {'Speed':>15}"
)
for step in range(max_steps):
    t0 = time.time()
    optimizer.zero_grad()
    accum = 0.0
    for micro_steps in range(grad_accum_step):
        x, y = trainloader.next_batch()
        x, y = x.to(device), y.to(device)
    
        with torch.autocast(device_type=device, dtype=torch.bfloat16):
            logits, loss = model(x, y)
        loss = loss / grad_accum_step
        accum += loss.detach()
        loss.backward()
    norm = torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

    lr = get_lr(step)
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr 
    
    optimizer.step()
    torch.cuda.synchronize()
    t1 = time.time()
    dt = (t1 - t0) * 1000
    tokensxsec = (trainloader.B * trainloader.T * grad_accum_step) / (t1 - t0)
    print(
        f"{step:6d} | {accum.item():8.4f} | {lr:10.2e} | {dt:9.1f} | {norm:10.4f} | {tokensxsec:15.2f} tokens/sec"
    )

Num decayed paramter tensor: 50, with 124,354,560 params
Num non decayed paramter tensor: 98, with 121,344 params
using fused AdamW: True
  Step |     Loss |         LR | Time (ms) |   GradNorm |           Speed
